# Análisis de estado de reservorios utilizando imagenes satelitales

## Dirección Nacional de Aguas (DINAGUA) - Ministerio de Ambiente

### Obtener la imagen Satelital de la Misión Sentinel-2

Importar los modulos de Python necesarios

In [162]:
# Import the packages
import ee
import geemap
import os
import geemap.colormaps as cm

In [ ]:
# Quitar el comentario para correr si no se esta inicializado y autenticado la app

# ee.Authenticate()
# ee.Initialize()

Generar el Mapa base centrado en Uruguay

In [163]:
Map = geemap.Map(center=[-32.7,-56], zoom=7)
Map

Map(center=[-32.7, -56], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

Para definir un área de interes, se debe realizar lo siguiente: 
1. Ir a la barra de digitalización ubicado en parte izquierda del mapa
2. Seleccionar una herramienta de digitalización. Por ejemplo, seleccionamos dibujar un rectangulo
3. Dibujamos el area de interes y cerramos el área. 

La siguiente linea de comando permite extraer el poligono determinado por el usuario y lo asigna a la variable roi 

In [164]:
Map.draw_features
roi = ee.FeatureCollection(Map.draw_features)

Definimos una fecha de inicio y final para buscar imagenes Sentinel Disponibles

In [165]:
startDate = '2023-03-01'
endDate = '2023-03-10'

Buscamos una colección de imagenes de la Misión Copernicus Sentinel-2. Ademas se realiza lo siguiente:
1. Filtrar por area de interes (roi)
2. Filtrar por la fecha de inicio y final

In [169]:
Collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(roi) \
    .filterDate(startDate, endDate)

En este punto se genera una lista con las imagenes satelitales, su correspondiente fecha y el porcentaje de nubes en el pixel 

In [175]:
# Obtener los porcentaje de nubes en la collección de imagenes
percentage_cloud = ee.List(Collection.aggregate_array('CLOUDY_PIXEL_PERCENTAGE')).getInfo()
# Obtener las fechas en que las imagenes fueron obtenidas
dates = ee.List(Collection \
    .aggregate_array('system:time_start')) \
    .map(lambda time_start: 
         ee.Date(time_start).format('dd-MMM-YYYY')
    ) \
    .getInfo()

Imprimir los resultados

In [184]:
# generar un indicador 
idx = 0
for cloud in percentage_cloud:
    print(f'La imagen tomada el {dates[idx]} tiene un porcentaje de nubes de {cloud}')
    idx =+ 1

La imagen tomada el 01-Mar-2023 tiene un porcentaje de nubes de 12.720492
La imagen tomada el 06-Mar-2023 tiene un porcentaje de nubes de 0.003275


Generamos una nueva collección con un filtro de Porcentaje de nubes menores a 10% 

In [182]:
Collection_cloud = Collection.filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",10)

Extraemos la imagen con menor porcentaje de nubes y extremos para el area de interes (roi). Esta imagen se agrega al mapa mostrado arriba

In [196]:
image = Collection_cloud.first().clip(roi)
# Parametros para graficar
Map.centerObject(roi)
vis_param = {'min': 228, 'max': 1705, 'bands': ['B4', 'B3', 'B2'], 'RGB': 1.5}
Map.addLayer(image, vis_param, "Color Verdadero")

Mostramos la fecha en que se tomó la imagen satelital

In [185]:
date = ee.Date(image.get('system:time_start'))
date2 = date.format('dd-MMM-YYYY')
print(f'La imagen corresponde al día {date2.getInfo()}')

La imagen corresponde al día 06-Mar-2023


### Calculo de indicadores 

En esta sección se calculará los siguientes indicadores
1. Automated Water Extraction Index (AWEI)
2. Normalized Difference Water Index (NDWI)
3. Modified Normalized Difference Water Index (MNDWI)

#### Automated Water Extraction Index (AWEI)

In [186]:
AWEI = image.expression(
  '4*(GREEN - SWIR1) - (0.25*NIR + 2.75*SWIR2)', {
      'GREEN': image.select('B3').multiply(0.0001),
      'NIR': image.select('B8').multiply(0.0001),
      'SWIR1': image.select('B11').multiply(0.0001),
      'SWIR2': image.select('B12').multiply(0.0001),})

#### Normalized Difference Water Index (NDWI)

In [187]:
NDWI = image.normalizedDifference(['B3', 'B8'])

Modified Normalized Difference Water Index (MNDWI)

In [188]:
MNDWI = image.normalizedDifference(["B8","B4"])

In [189]:
AWEI = AWEI.lt(0)
NDWI = NDWI.lt(0)
MNDWI = MNDWI.gt(0)

Combinar los indicadores para obtener un indice compuesto

In [190]:
waterCombine = AWEI.And(NDWI).And(MNDWI).rename('renameToSelectBands')

Generar una mascara para remover valores nulos

In [198]:
mask = waterCombine.eq(0)
water = waterCombine.updateMask(mask)

Obtener el área de agua disponible

Agregar la capa de Agua disponible en la imagen 

In [199]:
Map.centerObject(roi)
waterVisParam = {'palette':'29b0ff'}
Map.addLayer(water,waterVisParam, 'Agua Disponible')

In [200]:
Map.find_layer('Agua Disponible')

TileLayer(attribution='Google Earth Engine', max_zoom=24, name='Agua Disponible', options=['attribution', 'bou…